In [1]:
from emc.workflows.correct import init_emc_wf
from pathlib import Path
import nipype.pipeline.engine as pe
from nipype.interfaces import utility as niu
from dipy.data import get_fnames
from dmriprep.workflows.dwi.util import init_dwi_reference_wf
from dmriprep.interfaces.vectors import CheckGradientTable

#fdata, fbvals, fbvecs = get_fnames("sherbrooke_3shell")
fdata, fbvals, fbvecs = get_fnames("stanford_hardi")

In [2]:
# Create a workflow that can be added as a node to
# `~dmriprep.workflows.dwi.base.init_dwi_preproc_wf` and (for now) envelops
# `~dmriprep.workflows.dwi.util.init_dwi_reference_wf`
def build_emc_workflow(fdata, fbvals, fbvecs, work_dir='/tmp'):

    wf_emc = pe.Workflow(name='emc_correction')

    inputnode = pe.Node(niu.IdentityInterface(fields=['dwi_files', 'in_bval',
                                                      'in_bvec']),
        name='inputnode')

    inputnode.inputs.dwi_file = fdata
    inputnode.inputs.in_bval = fbvals
    inputnode.inputs.in_bvec = fbvecs

    emc_wf_node = init_emc_wf('emc_wf')

    wf_emc.connect([
        (inputnode, emc_wf_node, [('dwi_file', 'meta_inputnode.dwi_file'),
                                  ('in_bval', 'meta_inputnode.in_bval'),
                                  ('in_bvec', 'meta_inputnode.in_bvec')])
    ])

    wf_emc.base_dir = work_dir

    cfg = dict(execution={'stop_on_first_crash': False,
                          'parameterize_dirs': True, 'crashfile_format': 'txt',
                          'remove_unnecessary_outputs': False,
                          'poll_sleep_duration': 0, 'plugin': 'MultiProc'})
    for key in cfg.keys():
        for setting, value in cfg[key].items():
            wf_emc.config[key][setting] = value

    return wf_emc

In [ ]:
wf = build_emc_workflow(fdata, fbvals, fbvecs)
res = wf.run(plugin='MultiProc')

210223-16:08:13,453 nipype.workflow INFO:
	 Workflow emc_correction settings: ['check', 'execution', 'logging', 'monitoring']
210223-16:08:13,493 nipype.workflow INFO:
	 Running in parallel.
210223-16:08:13,497 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 4 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:13,598 nipype.workflow INFO:
	 [Job 0] Cached (emc_correction.emc_wf.split_dwis_node).
210223-16:08:13,609 nipype.workflow INFO:
	 [Job 1] Cached (emc_correction.emc_wf.emc_vectors_node).
210223-16:08:13,618 nipype.workflow INFO:
	 [Job 5] Cached (emc_correction.dwi_reference_wf.validate).
210223-16:08:13,621 nipype.workflow INFO:
	 [Job 6] Cached (emc_correction.gradient_table).
210223-16:08:13,625 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 3 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:13,725 nipype.workflow INFO:
	 [Job 2] Cached (emc_correction.emc_wf.prune_b0s_from_dwis_node).
210223-16:08

210223-16:08:15,722 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 12 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:15,807 nipype.workflow INFO:
	 [Job 26] Cached (emc_correction.emc_wf.enhance_and_skullstrip_template_mask_wf.initial_mask).
210223-16:08:15,812 nipype.workflow INFO:
	 [Job 31] Cached (emc_correction.emc_wf.enhance_and_skullstrip_template_mask_wf.rescale_image).
210223-16:08:15,816 nipype.workflow INFO:
	 [Job 91] Cached (_reg_0020).
210223-16:08:15,819 nipype.workflow INFO:
	 [Job 92] Cached (_reg_0021).
210223-16:08:15,823 nipype.workflow INFO:
	 [Job 93] Cached (_reg_0022).
210223-16:08:15,826 nipype.workflow INFO:
	 [Job 94] Cached (_reg_0023).
210223-16:08:15,829 nipype.workflow INFO:
	 [Job 95] Cached (_reg_0024).
210223-16:08:15,832 nipype.workflow INFO:
	 [Job 96] Cached (_reg_0025).
210223-16:08:15,835 nipype.workflow INFO:
	 [Job 97] Cached (_reg_0026).
210223-16:08:15,838 nipype.workflow INFO:
	 [Job 98] Cached (_reg

210223-16:08:16,851 nipype.workflow INFO:
	 [Job 142] Cached (_b0_based_image_transforms41).
210223-16:08:16,853 nipype.workflow INFO:
	 [Job 143] Cached (_b0_based_image_transforms42).
210223-16:08:16,856 nipype.workflow INFO:
	 [Job 144] Cached (_b0_based_image_transforms43).
210223-16:08:16,860 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 116 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:16,942 nipype.workflow INFO:
	 [Job 145] Cached (_b0_based_image_transforms44).
210223-16:08:16,946 nipype.workflow INFO:
	 [Job 146] Cached (_b0_based_image_transforms45).
210223-16:08:16,949 nipype.workflow INFO:
	 [Job 147] Cached (_b0_based_image_transforms46).
210223-16:08:16,952 nipype.workflow INFO:
	 [Job 148] Cached (_b0_based_image_transforms47).
210223-16:08:16,955 nipype.workflow INFO:
	 [Job 149] Cached (_b0_based_image_transforms48).
210223-16:08:16,958 nipype.workflow INFO:
	 [Job 150] Cached (_b0_based_image_transforms49).
210223-16:08:16

210223-16:08:17,665 nipype.workflow INFO:
	 [Job 219] Cached (_b0_based_image_transforms118).
210223-16:08:17,668 nipype.workflow INFO:
	 [Job 220] Cached (_b0_based_image_transforms119).
210223-16:08:17,671 nipype.workflow INFO:
	 [Job 221] Cached (_b0_based_image_transforms120).
210223-16:08:17,674 nipype.workflow INFO:
	 [Job 222] Cached (_b0_based_image_transforms121).
210223-16:08:17,677 nipype.workflow INFO:
	 [Job 223] Cached (_b0_based_image_transforms122).
210223-16:08:17,679 nipype.workflow INFO:
	 [Job 224] Cached (_b0_based_image_transforms123).
210223-16:08:17,681 nipype.workflow INFO:
	 [Job 225] Cached (_b0_based_image_transforms124).
210223-16:08:17,684 nipype.workflow INFO:
	 [Job 226] Cached (_b0_based_image_transforms125).
210223-16:08:17,687 nipype.workflow INFO:
	 [Job 227] Cached (_b0_based_image_transforms126).
210223-16:08:17,689 nipype.workflow INFO:
	 [Job 228] Cached (_b0_based_image_transforms127).
210223-16:08:17,693 nipype.workflow INFO:
	 [MultiProc] Runn

210223-16:08:20,972 nipype.workflow INFO:
	 [Job 299] Cached (_predict_dwis38).
210223-16:08:20,985 nipype.workflow INFO:
	 [Job 300] Cached (_predict_dwis39).
210223-16:08:20,998 nipype.workflow INFO:
	 [Job 301] Cached (_predict_dwis40).
210223-16:08:21,11 nipype.workflow INFO:
	 [Job 302] Cached (_predict_dwis41).
210223-16:08:21,24 nipype.workflow INFO:
	 [Job 303] Cached (_predict_dwis42).
210223-16:08:21,36 nipype.workflow INFO:
	 [Job 304] Cached (_predict_dwis43).
210223-16:08:21,49 nipype.workflow INFO:
	 [Job 305] Cached (_predict_dwis44).
210223-16:08:21,61 nipype.workflow INFO:
	 [Job 306] Cached (_predict_dwis45).
210223-16:08:21,74 nipype.workflow INFO:
	 [Job 307] Cached (_predict_dwis46).
210223-16:08:21,87 nipype.workflow INFO:
	 [Job 308] Cached (_predict_dwis47).
210223-16:08:21,92 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 102 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:21,186 nipype.workflow INFO:
	 [Job 309] Cached

210223-16:08:22,680 nipype.workflow INFO:
	 [Job 389] Cached (_predict_dwis128).
210223-16:08:22,692 nipype.workflow INFO:
	 [Job 390] Cached (_predict_dwis129).
210223-16:08:22,704 nipype.workflow INFO:
	 [Job 391] Cached (_predict_dwis130).
210223-16:08:22,716 nipype.workflow INFO:
	 [Job 392] Cached (_predict_dwis131).
210223-16:08:22,719 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 18 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:22,814 nipype.workflow INFO:
	 [Job 393] Cached (_predict_dwis132).
210223-16:08:22,827 nipype.workflow INFO:
	 [Job 394] Cached (_predict_dwis133).
210223-16:08:22,840 nipype.workflow INFO:
	 [Job 395] Cached (_predict_dwis134).
210223-16:08:22,853 nipype.workflow INFO:
	 [Job 396] Cached (_predict_dwis135).
210223-16:08:22,866 nipype.workflow INFO:
	 [Job 397] Cached (_predict_dwis136).
210223-16:08:22,878 nipype.workflow INFO:
	 [Job 398] Cached (_predict_dwis137).
210223-16:08:22,890 nipype.workflow INFO:
	

210223-16:08:24,41 nipype.workflow INFO:
	 [Job 469] Cached (_register_to_predicted58).
210223-16:08:24,44 nipype.workflow INFO:
	 [Job 470] Cached (_register_to_predicted59).
210223-16:08:24,49 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 90 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:24,136 nipype.workflow INFO:
	 [Job 471] Cached (_register_to_predicted60).
210223-16:08:24,140 nipype.workflow INFO:
	 [Job 472] Cached (_register_to_predicted61).
210223-16:08:24,143 nipype.workflow INFO:
	 [Job 473] Cached (_register_to_predicted62).
210223-16:08:24,146 nipype.workflow INFO:
	 [Job 474] Cached (_register_to_predicted63).
210223-16:08:24,149 nipype.workflow INFO:
	 [Job 475] Cached (_register_to_predicted64).
210223-16:08:24,152 nipype.workflow INFO:
	 [Job 476] Cached (_register_to_predicted65).
210223-16:08:24,155 nipype.workflow INFO:
	 [Job 477] Cached (_register_to_predicted66).
210223-16:08:24,158 nipype.workflow INFO:
	 [Job 478] C

210223-16:08:24,875 nipype.workflow INFO:
	 [Job 550] Cached (_register_to_predicted139).
210223-16:08:24,878 nipype.workflow INFO:
	 [Job 551] Cached (_register_to_predicted140).
210223-16:08:24,881 nipype.workflow INFO:
	 [Job 552] Cached (_register_to_predicted141).
210223-16:08:24,883 nipype.workflow INFO:
	 [Job 553] Cached (_register_to_predicted142).
210223-16:08:24,886 nipype.workflow INFO:
	 [Job 554] Cached (_register_to_predicted143).
210223-16:08:24,890 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs ready. Free memory (GB): 28.80/28.80, Free processors: 12/12.
210223-16:08:24,972 nipype.workflow INFO:
	 [Job 555] Cached (_register_to_predicted144).
210223-16:08:24,976 nipype.workflow INFO:
	 [Job 556] Cached (_register_to_predicted145).
210223-16:08:24,979 nipype.workflow INFO:
	 [Job 557] Cached (_register_to_predicted146).
210223-16:08:24,982 nipype.workflow INFO:
	 [Job 558] Cached (_register_to_predicted147).
210223-16:08:24,985 nipype.workflow INFO:
	 

210223-16:09:38,984 nipype.workflow INFO:
	 [MultiProc] Running 8 tasks, and 138 jobs ready. Free memory (GB): 27.20/28.80, Free processors: 4/12.
                     Currently running:
                       * _predict_dwis11
                       * _predict_dwis9
                       * _predict_dwis8
                       * _predict_dwis6
                       * _predict_dwis3
                       * _predict_dwis2
                       * _predict_dwis1
                       * _predict_dwis0
210223-16:09:40,295 nipype.workflow INFO:
	 [MultiProc] Running 12 tasks, and 134 jobs ready. Free memory (GB): 26.40/28.80, Free processors: 0/12.
                     Currently running:
                       * _predict_dwis15
                       * _predict_dwis14
                       * _predict_dwis13
                       * _predict_dwis12
                       * _predict_dwis11
                       * _predict_dwis9
                       * _predict_dwis8
                   